# Daily workflow - troselab (DRAFT)

`TR TODO: Jupyter ipywidget GUI`

(check Daniel's ipysheet mouse entry GUI for inspiration)

1. Upload to analysis server
    1. automatic (rcopy with file consolidation)?
    2. WinSCP?

    
2. Session ingest - Have this script parse the user folder. 
    1. retrieve login name  **DONE**
    2. parse user storage folder for SessionIDs, ScanIDs **DONE**
    3. intersect with 'Session' tables **DONE**
    4. FIRST STEP: session ingestion of new data  **DONE**
        1. have ipywidget with list of data to be ingested with checkbox  **DONE**
        2. have dropdowns next to SessionsFetch that expose 'Project', 'Anatomical Location', 'Equipment' (NOTE: have me or admin accept new entries? Or user?) - have a user default **DONE**
            1. ***TODO*** include a mini2p settings shorthand in elements `imaging.ProcessingParamSet`.
        3. IMPORTANT: Have dropdown for `Same_Location_as_Session` - default: same session
            1. Include `Same_Location_as_Session` key in `Session` table (default same session)
            2. Update after Session ingest
        4. have central 'START' button **DONE**
        5. Combine session entry with automatic RSpace entry (**DONE***)
            0. Include rspace in install requirements!
            1. Extract parent folder IDs under 'experiments' from RSpace (add RSpace API key and URL to djconfig )
            2. Intersect list with animal ID from session ingest
            3. Create new document with `YYYY-MM-DD_SessionID`
            4. Append content as verbose tables using Pandas df.to_html (`html_table = df.to_html()`)
            5. ***DONE*** IMPORTANT: Push RSpace document ID and noteobook URL in Sessions database!
        5. OPTIONAL: Have a notebook section to enter and insert Session Notes
        6. OPTIONAL: Submit Sessions to pyrat. Not sure if necessary.
        7. TODO: Think if splitting projects off to different database suffixes (and differnt associated tables makes sense from here. Seems reasonable given the focus on the user root directory here.
        8. ***DONE***: Implement auto animal pyrat ingest if needed. Crashes now.
        9. ***DONE***: Make sure that multi-scan sessions are properly treated. Currently they are not detected as such by the GUI. Make sure ingestion behavior is appropriate.


3. Scan ingest / 1st stage s2p processing: 
    1. ***NO!*** have another list with checkboxes (default checled) - now with all first stage scans of the newly ingested sessions. Have dropdown boxes with s2p parameteres. User default is selected. Followed by 'START'
    2. TODO: suppress s2p output, just have progress bars
    3. OPTIONAL: Have a notebook section to enter and insert Scan Notes
    4. OPTIONAL: Submit sessions, scans and Notes to RSPACE! Upload s2p average image, max image, Source image and Rastermap 
    5. Upload templates to rspace session, upload movie to rspace session
    

4. Scan Curation
    1. DIFFERENT NOTEBOOK (or different Notebook section)
    2. Make intersection of folder with database
        1. list _all_ scans
        2. have all non-curated red - the others green
        3. bhave non-processed scans grey
        4. have 'CURATE LOCALLY WITH s2P' buttons behind all scans. This button should spawn a local suite2p GUI which directly loads the respective suite2p stats.npy file from the server. Folder settings for server need to be set in datajoint.json file.

5. SERVER: Database backup and Data folder backup
    1. Have cronjob backup database to ana2 and isilon (midnight, incremental - only add, no delete. Increment ID to be reversible?)
    2. Have cronjob backup data folders to ana2 and isilon (midnight, incremental - only add, no delete)

## Login

Either log in via a local config file (see [01_pipeline](./01_pipeline.ipynb)), or enter login information manually. If you are don't have your login information, contact the administrator.


In [1]:
import os
# change to the upper level folder to detect dj_local_conf.json
if os.path.basename(os.getcwd())=='notebooks': os.chdir('..')
assert os.path.basename(os.getcwd())=='adamacs', ("Please move to the main directory")
from adamacs.pipeline import subject, session, equipment, surgery, event, trial, imaging
from adamacs.ingest import session as isess
import datajoint as dj
from rspace_client.eln import eln
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
dj.__version__

[2023-03-20 15:58:26,309][INFO]: Connecting tobiasr@172.26.128.53:3306
[2023-03-20 15:58:26,368][INFO]: Connected tobiasr@172.26.128.53:3306


'0.14.0'

### RSpace connection

In [2]:
URL=dj.config['custom'].get('rspace_URL')
API_KEY=dj.config['custom'].get('rspace_API_key')
api = eln.ELNClient(URL, API_KEY)
api.get_status()

{'message': 'OK', 'rspaceVersion': '1.80.1'}

## Activation
Next, import from `adamacs.pipeline` to activate the relevant schema.

In [3]:
from adamacs.utility import *
from adamacs.nbgui import *
from adamacs.pipeline import subject, session, surgery, scan, equipment

Assign easy names for relevant tables

In [4]:
sub, lab, protocol, line, mutation, user, project, subject_genotype, subject_death = (
    subject.Subject(), subject.Lab(), subject.Protocol(), subject.Line(), 
    subject.Mutation(), subject.User(), subject.Project(), subject.SubjectGenotype(), 
    subject.SubjectDeath()
    )

## Cleaning. Use with caution!

In [5]:
session.Session.delete()

[2023-03-20 15:58:30,817][INFO]: Deleting 0 rows from `tobiasr_session`.`session`
[2023-03-20 15:58:30,818][WARNING]: Nothing to delete.


0

## 1. Data upload
(not implemented currently)

## 2. Session ingest

### Define ingest GUI functions

In [14]:
# Tobias Rose 2023: Routine ingest helpers

import ipywidgets as widgets
from natsort import natsorted, ns
import re
from tqdm import tqdm

def select_sessions(IngestedSessionDirA, AvailableSessionDirB, Project, Recording_Location, Equipment, s2pparm, SessionNotes):
    
    # Personal default values
    user_defaults = get_user_defaults(AvailableSessionDirB)
       
    # Define the widgets
    session_dropdowns = []
    session_checkboxes = []
    for i, session_list in enumerate(AvailableSessionDirB):
        # Dropdowns for Project, Recording Location, Equipment

        current_session = get_session_key_from_dir([session_list])

        # check if directory session is already ingested - if yes: populate GUI with table values. If no: use user defaults
        query = session.Session() & f'session_id = "{current_session[0]}"'

        count = len(query.fetch('session_id'))
        # print(current_session)
        
        if count > 0:
            # print("HELLO")
            # get the project associated with a session
            query = session.ProjectSession() & f'session_id = "{current_session[0]}"'
            project_dropdown_value = query.fetch("project")
            # get the location associated with a session
            query = session.Session() * scan.ScanLocation() & f'session_id = "{current_session[0]}"'
            location_dropdown_value = query.fetch("anatomical_location")
            # get the equipment associated with a session
            query = session.Session() * scan.Scan() & f'session_id = "{current_session[0]}"'
            equipment_dropdown_value = query.fetch("scanner")
            # get the note associated with a session
            query = session.SessionNote() & f'session_id = "{current_session[0]}"'
            session_note_textbox_value = query.fetch("session_note")
            if len(session_note_textbox_value) == 0:
                session_note_textbox_value = ["none"]    
        else:
            project_dropdown_value = Project[user_defaults[i][0]]
            location_dropdown_value = [Recording_Location[user_defaults[i][1]]]
            equipment_dropdown_value = [Equipment[user_defaults[i][2]]]
            session_note_textbox_value = ["none"]
        
        project_dropdown = widgets.Dropdown(options=Project, value=project_dropdown_value, description="Project:")
        location_dropdown = widgets.Dropdown(options=Recording_Location, value=location_dropdown_value[0], description="Location:")
        equipment_dropdown = widgets.Dropdown(options=Equipment, value=equipment_dropdown_value[0], description="Setup:")
        s2pparms_dropdown = widgets.Dropdown(options=s2pparm[1], value=s2pparm[1][user_defaults[i][3]], description="s2p parm:")
        session_note_textbox = widgets.Text(value=session_note_textbox_value[0], description='Session comment:')

        session_dropdowns.append((project_dropdown, location_dropdown, equipment_dropdown, s2pparms_dropdown, session_note_textbox))

        # Checkbox for Process?
        session_checkbox = widgets.Checkbox(description='run?', layout=widgets.Layout(width='auto'))
        session_checkboxes.append(session_checkbox)

    # Display the widgets
    output = widgets.Output()

    with output:
        # Display the Sessions labels and associated dropdowns and checkboxes
        hbox_list = []
        for i, session_list in enumerate(AvailableSessionDirB): #unique_directory_strings(SessionDirA, SessionDirB)
            # Create an HBox to hold the label and associated dropdowns and checkbox
            hbox = widgets.HBox()
            hbox.children = [
                widgets.Label(value=session_list + ':', layout=widgets.Layout(width='1800px')), 
                session_dropdowns[i][0],
                session_dropdowns[i][1],
                session_dropdowns[i][2],
                session_dropdowns[i][3],
                session_dropdowns[i][4],
                session_checkboxes[i]
            ]
            if session_list in unique_directory_strings(IngestedSessionDirA, AvailableSessionDirB):
                hbox.children[0].value = '*' + session_list + ':'
                hbox.children[6].value = True
            else:
                hbox.children[6].value = False
                hbox.children[1].disabled = True
                hbox.children[2].disabled = True
                hbox.children[3].disabled = True
                hbox.children[4].disabled = True
                hbox.children[5].disabled = True
            hbox_list.append(hbox)

        vbox = widgets.VBox(hbox_list, layout=widgets.Layout(flex='0 0 auto', overflow_y='scroll'))
        # Display the commit button
        commit_button = widgets.Button(description='Commit', layout=widgets.Layout(width='auto'))
        display(vbox, commit_button)

        # Define the callback function for the commit button
        def commit_button_clicked(b):
            selected_sessions = [AvailableSessionDirB[i] for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_projects = [session_dropdowns[i][0].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_locations = [session_dropdowns[i][1].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_equipment = [session_dropdowns[i][2].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            selected_s2pparms = [session_dropdowns[i][3].index for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]
            entered_session_note = [session_dropdowns[i][4].value for i in range(len(AvailableSessionDirB)) if session_checkboxes[i].value]

            selected_scans = get_scan_key_from_dir(selected_sessions)
            selected_sessions = get_session_key_from_dir(selected_sessions)

            selected_s2pparms_index = s2pparm[0][selected_s2pparms]

            output.clear_output()
            with output:
                # print('Selected Sessions:', selected_sessions)
                # print('Selected Scans:', selected_scans)
                # print('Selected Projects:', selected_projects)
                # print('Selected Recording Locations:', selected_locations)
                # print('Selected Equipment:', selected_equipment)
                # print('Selected S2PParms:', selected_s2pparms)

                # Ingest selected sessions here
                populate_settings = {'display_progress': True, 'suppress_errors': False, 'processes': 0}

                for i, sessi in enumerate(tqdm(selected_sessions, desc='Current Session: {}')):
                    # ingest sessions
                    isess.ingest_session_scan(sessi, verbose=False, project_key=selected_projects[i], equipment_key=selected_equipment[i], location_key=selected_locations[i], software_key='ScanImage')
                    # update / insert session info based on user choice from above
                    try:
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    except:
                        session.SessionNote.delete_quick({'session_id': sessi})
                        session.SessionNote.insert1({'session_id': sessi, 'session_note': entered_session_note[i]})
                    
                    # update scaninfo based on user choice from above
                    # get the scans associated with a session
                    query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
                    scans_to_process = query.fetch("scan_id")

                    for j, scansi in enumerate(scans_to_process):
                        try:
                            scan.Scan.update1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        except:
                            scan.Scan.insert1({'session_id': sessi, 'scan_id': scansi, 'scan_notes': entered_session_note[i]})
                        try:
                            scan.ScanLocation.update1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        except: 
                            scan.ScanLocation.insert1({'session_id': sessi, 'scan_id': scansi, 'anatomical_location': selected_locations[i]})
                        
                        # print('Ingested and processed Scan:', scansi)
                    
                    # SCAN Processing 
                    scan.ScanInfo.populate(**populate_settings)

                    for j, scansi in enumerate(scans_to_process):
                        query = session.SessionDirectory() & 'session_id = "' + sessi + '"'
                        print(query)
                        dir_proc = query.fetch('session_dir')[0]
                    imaging.ProcessingTask.insert1((sessi, scansi, 3, dir_proc, 'trigger'), skip_duplicates=True)
                    ## Make RSpace entries 

                    query = session.Session() * subject.User() & f'session_id = "{sessi}"'
                    animalID = query.fetch("subject")[0]
                    date = query.fetch("session_datetime")[0].strftime("%Y-%m-%d")
                    userID = query.fetch("initials")[0]
                    sessionID = query.fetch("session_id")[0]
                    fetchtable = session.Session() * scan.ScanPath() * session.SessionUser() * session.ProjectSession() * session.SessionNote() * scan.Scan() & f'session_id = "{sessi}"'

                    make_rspace_session_document(animalID, sessi, date, userID, fetchtable) # TODOO
                
                
                print('Ingested Sessions:', selected_sessions)

        # Attach the callback function to the commit button
        commit_button.on_click(commit_button_clicked)

    # Display the output
    display(output)
        
def make_rspace_session_document(animalID, sessionID, date, userID, fetchtable):

    # find 'Experiments' folder and get ID
    folders = api.list_folder_tree()
    experiments_ids = []
    for record in folders['records']:
        if record['name'] == 'Experiments':
            experiments_ids.append(record['id'])

    # find folders under 'Experiments' folder
    names_and_ids = []
    subfolders = api.list_folder_tree(experiments_ids[0])
    names_and_ids.append([(record['name'], record['id']) for record in subfolders['records']])

    # find matches with animal IDs
    match_found = any(animalID in item[0] for sublist in names_and_ids for item in sublist)

    # if found, find folder id matching that animal
    if match_found:
        save_folder_id = [x[1] for sublist in names_and_ids for x in sublist if animalID in x[0]][0]
    else:
        # if not generate the directory
        new_folder = api.create_folder(name=f"{userID}_{animalID}", parent_folder_id=experiments_ids[0], notebook=True)
        save_folder_id = new_folder['id']


    # find matches with document IDs

    # find all documents in the animal folder
    all_animal_documents = api.list_folder_tree(save_folder_id)
    doc_names_and_ids = []
    doc_names_and_ids.append([(record['name'], record['id']) for record in all_animal_documents['records']])
    # find matches with sessionID
    match_found = any(sessionID in item[0] for sublist in doc_names_and_ids for item in sublist)

    if match_found:
        ids = [id for string, id in doc_names_and_ids[0] if sessionID in string]
        new_doc = {"id": ids[0]}
    else:
        new_doc = api.create_document(name=f"{date}_{sessionID}", parent_folder_id=save_folder_id)

    # create RSpace document
    df = fetchtable.fetch(format='frame')
    html_table = df.to_html()
    content = html_table
    api.append_content(new_doc['id'], content)

    # ingest document in rspace table
    session.SessionRspace.insert1({'session_id': sessionID, 'rspace_id': new_doc['id']}, skip_duplicates=True)
    # ingest animal folder id in subject Rspace table
    subject.SubjectRspace.insert1({'subject': animalID, 'rspace_subject_id': save_folder_id}, skip_duplicates=True)
    # session.SessionRspace.insert1({'rspace_id': new_doc['id'], 'rspace_url': new_doc['_links'][0]['link']})
def get_date_key_from_dir(directory):
    return directory.split("_")[-1]

def get_session_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]
     
def get_scan_dir_key_from_dir(directory):
    return [path.split('/')[-1] for path in directory]

def get_session_key_from_dir(string):
    result = [re.search(r'sess\S+', item).group(0) for item in string]
    return result

def get_user_initials_from_dir(string):
    result = [name[:2] for name in string]
    return result

def get_scan_key_from_dir(string):
    result = [re.search(r'scan\S+_', item).group(0)[:-1] for item in string]
    return result

def unique_directory_strings(dirs1, dirs2):
    set1 = set(dirs1)
    set2 = set(dirs2)
    common_dirs = list(set1.intersection(set2))
    unique_dirs = list(set(set1.union(set2)) - set(common_dirs))
    return unique_dirs

def get_user_defaults(directory):
    RN = [2, 2, 0, 2]
    JJ = [2, 6, 2, 1]
    TR = [2, 2, 0, 2]
    LK = [7, 1, 0, 2]
    DB = [4, 0, 1, 1]
    NK = [6, 5, 6, 1]
    
    user_initials = get_user_initials_from_dir(directory)

    user_arrays = {}
    for initial in user_initials:
        values = locals()[initial]
        user_arrays[initial] = values

    new_array = [user_arrays[initial] for initial in user_initials]
    return new_array

#### Populate ingest GUI variables from DJ tables and manual arrays

In [7]:
# DJ tables
Project = project.fetch('project')
Equipment = equipment.Equipment().fetch('scanner')
Recording_Location = surgery.AnatomicalLocation().fetch('anatomical_location')

session_dirs_ingested = session.SessionDirectory.fetch('session_dir')
scan_dirs_ingested = scan.ScanPath.fetch('path')
SessionsDirArrayingested = get_session_dir_key_from_dir(session_dirs_ingested)
ScanDirArrayingested = get_scan_dir_key_from_dir(scan_dirs_ingested)

SessionNotes = session.SessionNote.fetch('session_note')

# Manual arrays
s2pparm = imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")
# ['mini2p_a', 'mini2p_b', 'bench2p_a']
# s2pparm[0][2]÷

In [8]:
imaging.ProcessingParamSet.fetch("paramset_idx", "paramset_desc")

[array([1, 2, 3]),
 array(['TR: Mini2p (single channel, single plane, non-rigid, 15Hz)',
        'TR: Bench2p (single channel, single plane, non-rigid, 30Hz)',
        'TR: Bench2p (single channel, single plane, rigid, 30Hz)'],
       dtype=object)]

In [9]:
imaging.ProcessingParamSet()

paramset_idx,processing_method,paramset_desc,param_set_hash,params dictionary of all applicable parameters
1,suite2p,"TR: Mini2p (single channel, single plane, non-rigid, 15Hz)",e92bc8e7-0ef9-e5f6-78e3-b5557d7bfd9e,=BLOB=
2,suite2p,"TR: Bench2p (single channel, single plane, non-rigid, 30Hz)",872cc04a-2de9-9002-df20-413b86d64f2c,=BLOB=
3,suite2p,"TR: Bench2p (single channel, single plane, rigid, 30Hz)",73146edd-a587-2e17-2065-42b628fcc019,=BLOB=


#### Parse user data root dir and sort by date-string in directory name

In [10]:
dataroot = dj.config['custom']['exp_root_data_dir'][0]
dirs_root = [d for d in os.listdir(dataroot) if os.path.isdir(os.path.join(dataroot, d)) and 'sess' in d]
sorted_dirs_root = natsorted(dirs_root, key=get_date_key_from_dir, reverse = True)

### Generate Ingest GUI

In [13]:
# print("\033[1m" + 'Select:                                           Project        Location  Setup        s2p parm' + "\033[0m" )
print("\033[1m" + 'INGEST GUI' + "\033[0m" )
selected_data = select_sessions(SessionsDirArrayingested, sorted_dirs_root, Project, Recording_Location, Equipment, s2pparm, SessionNotes)

INGEST GUI


Output()

In [15]:
scan.ScanInfo()

session_id,scan_id,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FB2LN5C,scan9FB2LN5C,1,2,1,5000,0,0.0,0.0,0.0,15.3846,1,250.0,0.712867,None,325.0
sess9FCJJS40,scan9FCJJS40,1,2,1,9000,0,0.0,0.0,1790.78,15.3845,1,250.003,0.95,None,585.004
sess9FF6TAW6,scan9FF6TAW6,1,2,1,9250,0,-51109800.0,-5622000.0,10147.9,15.3845,1,250.003,0.95,None,601.255
sess9FGLE1FN,scan9FGLE1FN,1,1,1,27000,0,39113.7,18761.0,9571.63,29.9784,1,63.1769,0.712867,None,900.648
sess9FGLEFJ3,scan9FGLEFJ3,1,1,1,27000,0,39113.2,18761.0,9571.63,29.9784,1,63.1769,0.712867,None,900.648
sess9FGLZLRI,scan9FGLZLRI,1,1,1,5000,0,-2687.11,-30464600.0,-123841000.0,15.3845,1,250.001,0.95,None,325.002


In [16]:
scan.Scan  * scan.ScanInfo() * scan.ScanPath()

session_id,scan_id,user_id,path,scanner,acq_software,scan_notes free-notes,nfields number of fields,nchannels number of channels,ndepths Number of scanning depths (planes),nframes number of recorded frames,nrois number of ROIs (see scanimage's multi ROI imaging),x (um) ScanImage's 0 point in the motor coordinate system,y (um) ScanImage's 0 point in the motor coordinate system,z (um) ScanImage's 0 point in the motor coordinate system,fps (Hz) frames per second - Volumetric Scan Rate,bidirectional true = bidirectional scanning,usecs_per_line microseconds per scan line,fill_fraction raster scan temporal fill fraction (see scanimage),scan_datetime datetime of the scan,scan_duration (seconds) duration of the scan
sess9FB2LN5C,scan9FB2LN5C,4,/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C,dummy,ScanImage,none,1,2,1,5000,0,0.0,0.0,0.0,15.3846,1,250.0,0.712867,None,325.0
sess9FCJJS40,scan9FCJJS40,6,/datajoint-data/data/tobiasr/JJ_ROS-1346_2022-06-15_scan9FCJJS40_sess9FCJJS40,macroscope,ScanImage,none,1,2,1,9000,0,0.0,0.0,1790.78,15.3845,1,250.003,0.95,None,585.004
sess9FF6TAW6,scan9FF6TAW6,6,/datajoint-data/data/tobiasr/JJ_ROS-1438_2022-11-22_15_35scan9FF6TAW6_sess9FF6TAW6,macroscope,ScanImage,none,1,2,1,9250,0,-51109800.0,-5622000.0,10147.9,15.3845,1,250.003,0.95,None,601.255
sess9FGLE1FN,scan9FGLE1FN,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLE1FN_sess9FGLE1FN,bench2p,ScanImage,none,1,1,1,27000,0,39113.7,18761.0,9571.63,29.9784,1,63.1769,0.712867,None,900.648
sess9FGLEFJ3,scan9FGLEFJ3,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3,bench2p,ScanImage,none,1,1,1,27000,0,39113.2,18761.0,9571.63,29.9784,1,63.1769,0.712867,None,900.648
sess9FGLZLRI,scan9FGLZLRI,7,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,bench2p,ScanImage,none,1,1,1,5000,0,-2687.11,-30464600.0,-123841000.0,15.3845,1,250.001,0.95,None,325.002


## Run imaging processing tasks

In [17]:
imaging.ProcessingTask()

session_id,scan_id,paramset_idx,processing_output_dir output directory of the processed scan relative to root data directory,"task_mode 'load': load computed analysis results, 'trigger': trigger computation"
sess9FB2LN5C,scan9FB2LN5C,3,/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C,trigger
sess9FCJJS40,scan9FCJJS40,3,/datajoint-data/data/tobiasr/JJ_ROS-1346_2022-06-15_scan9FCJJS40_sess9FCJJS40,trigger
sess9FF6TAW6,scan9FF6TAW6,3,/datajoint-data/data/tobiasr/JJ_ROS-1438_2022-11-22_15_35scan9FF6TAW6_sess9FF6TAW6,trigger
sess9FGLE1FN,scan9FGLE1FN,3,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLE1FN_sess9FGLE1FN,trigger
sess9FGLEFJ3,scan9FGLEFJ3,3,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-15_scan9FGLEFJ3_sess9FGLEFJ3,trigger
sess9FGLZLRI,scan9FGLZLRI,3,/datajoint-data/data/tobiasr/RN_OPI-1681_2023-02-16_scan9FGLZLRI_sess9FGLZLRI,trigger


In [18]:
imaging.Processing.populate(display_progress=False,  suppress_errors=True)

Processing:   0%|          | 0/6 [00:00<?, ?it/s]

using TorchFFT
{'data_path': ['/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C'], 'tiff_list': ['/datajoint-data/data/tobiasr/DB_WEZ-8701_2022-03-18_scan9FB2LN5C_sess9FB2LN5C/scan9FB2LN5C_DB_WEZ-8701_02027_00001.tif']}
tif
** Found 1 tifs - converting to binary **
time 3.05 sec. Wrote 10000 frames per binary for 1 planes
>>>>>>>>>>>>>>>>>>>>> PLANE 0 <<<<<<<<<<<<<<<<<<<<<<
NOTE: not registered / registration forced with ops['do_registration']>1
      (no previous offsets to delete)
NOTE: Applying builtin classifier at /home/tobiasr/suite2p/suite2p/classifiers/classifier.npy
----------- REGISTRATION
NOTE: estimated bidiphase offset from data: -7 pixels
Reference frame, 5.71 sec.
Registered 8000/10000 in 9.53s
Registered 10000/10000 in 11.83s
----------- Total 23.31 sec
Registration metrics, 5.49 sec.
----------- ROI DETECTION
Binning movie in chunks of length 15
Binned movie of size [564,208,208] created in 1.16 sec.
Binned movie denoised (for cell detectio

Processing:   0%|          | 0/6 [00:46<?, ?it/s]

NOTE: estimated spatial scale ~12 pixels, time epochs 1.00, threshold 10.00 
Detected 0 ROIs, 0.68 sec


ValueError: no ROIs were found -- check registered binary and maybe change spatial scale

### Do manual suite2p curation now!
### Then: populate curation table

In [ ]:
imaging.Curation().create1_from_processing_task({'session_id': "sess9FGLE1FN", 'scan_id': "scan9FGLE1FN", "paramset_idx":3})

In [ ]:
imaging.Curation()

TODO: 
- figure out how to re-run s2p after curation


Populate imaging.MotionCorrection

In [ ]:
populate_settings = {'display_progress': False, 'suppress_errors': False, 'processes': 1}
imaging.MotionCorrection.populate(**populate_settings)

In [ ]:
imaging.MotionCorrection()

In [ ]:
imaging.Segmentation.populate(**populate_settings)

In [ ]:
imaging.Segmentation()

In [ ]:
imaging.MaskClassification.populate(**populate_settings)

In [ ]:
imaging.MaskClassification()

In [ ]:
imaging.Fluorescence.populate(**populate_settings)

In [ ]:
imaging.Fluorescence()

In [ ]:
imaging.Activity.populate(**populate_settings)

In [ ]:
imaging.Activity()

## Exploring data

In [ ]:
session_key = (session.Session & 'session_id = "sess9FGLE1FN"').fetch('KEY')[0]
session_key

In [ ]:
scan.ScanInfo & session_key

scan.ScanInfo.Field & session_key

In [ ]:
imaging.ProcessingTask * imaging.Processing & session_key

In [ ]:
imaging.Curation & session_key

In [ ]:
curation_key = (imaging.Curation & session_key & 'curation_id=1').fetch1('KEY')
curation_key

In [ ]:
imaging.MotionCorrection.Summary()

In [ ]:
average_image = (imaging.MotionCorrection.Summary & curation_key & 'field_idx=0').fetch1('average_image')
plt.imshow(average_image)

In [ ]:
mask_xpix, mask_ypix = (imaging.Segmentation.Mask * imaging.MaskClassification.MaskType & curation_key & 'mask_center_z=0' & 'mask_npix > 130').fetch('mask_xpix','mask_ypix')

mask_image = np.zeros(np.shape(average_image), dtype=bool)
for xpix, ypix in zip(mask_xpix, mask_ypix):
    mask_image[ypix, xpix] = True

plt.imshow(average_image);
plt.contour(mask_image, colors='white', linewidths=0.5);

In [ ]:
query_cells = (imaging.Segmentation.Mask * imaging.MaskClassification.MaskType & curation_key & 'mask_center_z=0' & 'mask_npix > 130').proj()

fluorescence_traces = (imaging.Fluorescence.Trace & query_cells).fetch('fluorescence', order_by='mask')

activity_traces = (imaging.Activity.Trace & query_cells).fetch('activity_trace', order_by='mask')

sampling_rate = (scan.ScanInfo & curation_key).fetch1('fps') # [Hz]

fig, ax = plt.subplots(1, 1, figsize=(16, 4))
ax2 = ax.twinx()

for f, a in zip(fluorescence_traces, activity_traces):
    ax.plot(np.r_[:f.size] * 1/sampling_rate, f, 'w', label='fluorescence trace')    
    ax2.plot(np.r_[:a.size] * 1/sampling_rate, a, 'r', alpha=0.5, label='deconvolved trace')
    
    break

ax.tick_params(labelsize=14)
ax2.tick_params(labelsize=14)

ax.legend(loc='upper left', prop={'size': 14})
ax2.legend(loc='upper right', prop={'size': 14})

ax.set_xlabel('Time (s)')
ax.set_ylabel('Activity (a.u.)')
ax2.set_ylabel('Activity (a.u.)');

In [ ]:
# plt.imshow(zip(fluorescence_traces))fluorescence_traces
allF = np.vstack(fluorescence_traces

In [ ]:
plt.imshow(allF)

## Exploring updates

In [ ]:
dj.Diagram(subject.Subject) + dj.Diagram(session.Session) + dj.Diagram(scan) + dj.Diagram(imaging.Processing) + dj.Diagram(imaging.Curation)

In [ ]:
from element_calcium_imaging.plotting.widget import main

selected_data

In [ ]:
sessi = "sess9FG1TAXY"
query = session.Session() * scan.Scan() & f'session_id = "{sessi}"'
scans_to_process = query.fetch("scan_id")
for j, scansi in enumerate(scans_to_process):
    print(scansi)

In [ ]:
session.Session() * session.SessionRspace()

In [ ]:
subject.Subject.delete()

### 2.1 Session notes ingest

In [ ]:
current_session = "sess9FGTSPEH"
query = session.Session() * scan.Scan() & f'session_id = "{current_session}"'
equipment_dropdown_value = query.fetch("scanner")
equipment_dropdown_value

## 3. Scan ingest / s2p processing 1st pass

In [ ]:
query = session.Session() * scan.ScanLocation() & f'session_id = "{current_session}"'

In [ ]:
query